In [1]:
from models import *
from utils import set_seed
from data.loaders import *
from utils import *

In [2]:
user = 'ajimenez'
path_to_model = f"/home/{user}/CoSE-pytorch/wandb/latest-run/files/weights_trained/epoch_150"

In [3]:
set_seed(0)
device = torch.device('cuda:0')
cose = CoSEModel('config.json', False)
cose.encoder.load_state_dict(torch.load(os.path.join(os.getcwd(), path_to_model,"encoder.pth"), map_location=device))
cose.decoder.load_state_dict(torch.load(os.path.join(os.getcwd(),path_to_model ,"decoder.pth"), map_location=device))
cose.position_predictive_model.load_state_dict(torch.load(os.path.join(os.getcwd(),path_to_model,"pos_pred.pth"), map_location=device))
cose.embedding_predictive_model.load_state_dict(torch.load(os.path.join(os.getcwd(),path_to_model,"emb_pred.pth"), map_location=device))

<All keys matched successfully>

In [4]:
cose.encoder = cose.encoder.eval()
cose.decoder = cose.decoder.eval()
cose.embedding_predictive_model = cose.embedding_predictive_model.eval()
cose.position_predictive_model = cose.position_predictive_model.eval()

In [5]:
val_path = f"/data/{user}/cose/test/"

In [6]:
batchdata = BatchCoSELoader(path = val_path,
                    filenames={"inputs_file" : "inputs_list_based.pkl",
                               "targets_file": "target_list_based.pkl"
                              }
                )

valid_loader = DataLoader(
                    dataset =batchdata,
                    batch_size = 1, #data is already in batch mode, batch_size = 1 means iterating every .get_next() returns a new batch
                )
stats_json = 'didi_wo_text-stats-origin_abs_pos.json'
stats_path = '/data/jcabrera/didi_wo_text/'
with open(os.path.join(stats_path, stats_json)) as json_file:
    stats = json.load(json_file)

mean_channel = stats['mean_channel'][:2]
std_channel = np.sqrt(stats['var_channel'][:2])
log_dir = f'/home/ajimenez/pruebas'

In [7]:
i = 0
for batch_input, batch_target in iter(valid_loader):
    if i == 1:
        break
    i+=1

In [8]:
eval_loss = AggregateAvg()

In [9]:
out_eval_parse_input = eval_parse_input(batch_input, cose.device)
out_eval_parse_target = eval_parse_target(batch_target, cose.device)
encoder_inputs, _, strok_len_inputs, _, _ = out_eval_parse_input
# passing inputs to encoding
comb_mask, look_ahead_mask, _ = generate_3d_mask(encoder_inputs, strok_len_inputs,cose.device, cose.config.enc_nhead)
encoder_out = cose.encoder(encoder_inputs.permute(1,0,2), strok_len_inputs, comb_mask)

In [10]:
models_tuple = [cose.embedding_predictive_model, cose.decoder]
stats_tuple = [mean_channel, std_channel]

In [11]:
eval_loss, recon_chamfer, pred_chamfer = quantitative_eval_step(encoder_out, out_eval_parse_input, out_eval_parse_target, models_tuple, stats_tuple, eval_loss, cose.device, cose.config.rel_nhead )

/home/ajimenez/CoSE-pytorch/models/gmm.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out_pi = torch.nn.functional.softmax(out_pi)


In [12]:
eval_loss